In [7]:
from pathlib import Path
import os
import pandas as pd
from datetime import datetime

def _safe_write_csv(df: pd.DataFrame, out_path: Path) -> Path:
    """
    安全写入：先写临时文件，再尝试原子替换；失败则落盘到备用文件名。
    返回最终写入的路径。
    """
    out_path.parent.mkdir(parents=True, exist_ok=True)
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    tmp_path = out_path.with_name(f"{out_path.stem}.tmp_{os.getpid()}_{ts}{out_path.suffix or '.csv'}")
    alt_path = out_path.with_name(f"{out_path.stem}_alt_{ts}{out_path.suffix or '.csv'}")

    # 写入临时文件
    df.to_csv(tmp_path, index=False)

    # 尝试原子替换
    try:
        os.replace(tmp_path, out_path)  # Windows 原子替换，若目标被占用会报 PermissionError
        print(f"处理后的数据已成功保存到: {out_path}")
        return out_path
    except PermissionError:
        # 目标被占用或无权限，退而求其次：改写到备用文件
        os.replace(tmp_path, alt_path)
        print(f"警告: 无法写入 {out_path}（可能被占用或只读）。已改写到: {alt_path}")
        return alt_path

def mark_and_save_data(input_path: str, output_path: str):
    """
    加载期权历史数据，解析合约名称，标记活跃合约，并保存结果（含安全写入与降级逻辑）。
    """
    df = pd.read_csv(input_path)
    print(f"原始数据加载成功，共 {len(df)} 条记录。")

    parts = df["instrument_name"].str.split("-", expand=True)
    df["underlying"] = parts[0] + "-" + parts[1]
    df["expiration"] = pd.to_datetime(parts[2], format="%y%m%d")
    df["strike_price"] = parts[3].astype(float)
    df["option_type"] = parts[4]

    df["is_active"] = df["vol"] > 0
    active_count = df["is_active"].sum()
    print(f"完成活跃度标记，共 {active_count} 条记录被标记为活跃。")

    final_path = _safe_write_csv(df, Path(output_path))
    return str(final_path)

def find_all_opportunities(marked_data_path: str):
    """
    扫描所有时间点的蝶式机会。
    """
    print("--- 开始通用扫描：查找所有组合在所有时间点的机会 ---")
    try:
        df = pd.read_csv(marked_data_path, parse_dates=["expiration"])
    except FileNotFoundError:
        print(f"错误: 找不到标记好的数据文件 '{marked_data_path}'。")
        return

    df["readable_time"] = pd.to_datetime(df["open_time"], unit="ms")
    opportunities_found = 0
    grouped = df.groupby("open_time")
    EPSILON = 1e-9

    print(f"数据中包含 {len(grouped)} 个独立的时间点，将开始逐一扫描...")

    for _, group_df in grouped:
        df_sorted = group_df.sort_values(by=["expiration", "option_type", "strike_price"]).reset_index(drop=True)
        if len(df_sorted) < 3:
            continue

        for i in range(1, len(df_sorted) - 1):
            row_a, row_b, row_c = df_sorted.iloc[i - 1], df_sorted.iloc[i], df_sorted.iloc[i + 1]

            if not (row_a["expiration"] == row_b["expiration"] == row_c["expiration"]
                    and row_a["option_type"] == row_b["option_type"] == row_c["option_type"]):
                continue

            strike_a, strike_b, strike_c = row_a["strike_price"], row_b["strike_price"], row_c["strike_price"]
            if not (abs((strike_b - strike_a) - (strike_c - strike_b)) < EPSILON and (strike_b - strike_a) > 0):
                continue

            if not (row_a["is_active"] and row_b["is_active"] and row_c["is_active"]):
                continue

            close_a, close_b, close_c = row_a["close"], row_b["close"], row_c["close"]
            if (2 * close_b) > (close_a + close_c + EPSILON):
                opportunities_found += 1
                print("\n!!! 发现潜在机会 !!!")
                print(f"  时间点: {row_b['readable_time']}")
                print(f"  中间合约 (B): {row_b['instrument_name']} (Close: {close_b})")
                print(f"  相邻合约 (A): {row_a['instrument_name']} (Close: {close_a})")
                print(f"  相邻合约 (C): {row_c['instrument_name']} (Close: {close_c})")
                print(f"  满足条件: 2 * {close_b} > {close_a} + {close_c}  ({2*close_b:.6f} > {close_a + close_c:.6f})")

    if opportunities_found == 0:
        print("\n扫描完成，未发现满足条件的机会。")
    else:
        print(f"\n扫描完成，共发现 {opportunities_found} 个潜在机会。")

if __name__ == "__main__":
    raw_data_file = "data/BTC-USD-optionchain-candlesticks-2025-10-09.csv"
    marked_data_file = "data/options_data_marked.csv"

    final_marked = mark_and_save_data(raw_data_file, marked_data_file)
    find_all_opportunities(final_marked)

原始数据加载成功，共 808544 条记录。
完成活跃度标记，共 5441 条记录被标记为活跃。
警告: 无法写入 data\options_data_marked.csv（可能被占用或只读）。已改写到: data\options_data_marked_alt_20251022_140938.csv
--- 开始通用扫描：查找所有组合在所有时间点的机会 ---
数据中包含 1440 个独立的时间点，将开始逐一扫描...

!!! 发现潜在机会 !!!
  时间点: 2025-10-08 17:10:00
  中间合约 (B): BTC-USD-251009-124000-C (Close: 0.0042)
  相邻合约 (A): BTC-USD-251009-123500-C (Close: 0.006)
  相邻合约 (C): BTC-USD-251009-124500-C (Close: 0.0022)
  满足条件: 2 * 0.0042 > 0.006 + 0.0022  (0.008400 > 0.008200)

!!! 发现潜在机会 !!!
  时间点: 2025-10-09 08:05:00
  中间合约 (B): BTC-USD-251010-122500-C (Close: 0.0055)
  相邻合约 (A): BTC-USD-251010-122000-C (Close: 0.007)
  相邻合约 (C): BTC-USD-251010-123000-C (Close: 0.0038)
  满足条件: 2 * 0.0055 > 0.007 + 0.0038  (0.011000 > 0.010800)

!!! 发现潜在机会 !!!
  时间点: 2025-10-09 08:58:00
  中间合约 (B): BTC-USD-251010-128000-C (Close: 0.0002)
  相邻合约 (A): BTC-USD-251010-127000-C (Close: 0.0002)
  相邻合约 (C): BTC-USD-251010-129000-C (Close: 0.0001)
  满足条件: 2 * 0.0002 > 0.0002 + 0.0001  (0.000400 > 0.000300)

!!! 发现潜在机会 !!